In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.neighbors import kneighbors_graph
from scipy import sparse

# ------------------------------------
# Configuration & Hyperparameters
# ------------------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEEDS = [8,74,88,94,111]
KNN_K = 5
HEAT_KERNEL_T = 1.0

SAGE_HIDDEN_DIM = 128
SAGE_DROPOUT = 0.2
LR = 4e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 15000
PATIENCE = 1000

# ------------------------------------
# Build Weighted KNN Graph
# ------------------------------------
def build_knn_graph(X: torch.Tensor, k: int, t: float = 1.0) -> np.ndarray:
    X_np = X.cpu().numpy()
    W = kneighbors_graph(X_np, k, mode='distance', include_self=False)
    W = W.maximum(W.T)  # make symmetric

    # Apply heat kernel weights: exp(-d^2 / 4t)
    W.data = np.exp(-W.data ** 2 / (4 * t))

    # Convert to binary adjacency matrix for edge_index
    adj = (W > 0).astype(int).toarray()

    return adj

# ------------------------------------
# GNN Model
# ------------------------------------
class GraphSAGENet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.out = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv3(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv4(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        return self.out(x, edge_index)

# ------------------------------------
# Train and Evaluate for One Seed
# ------------------------------------
def run_single_seed(X_raw, Y_raw, seed_model):
    np.random.seed(seed_model)
    torch.manual_seed(seed_model)
    if DEVICE.type == 'cuda':
        torch.cuda.manual_seed_all(seed_model)

    scaler = StandardScaler()
    X_np = scaler.fit_transform(np.abs(X_raw.cpu().numpy()))
    X = torch.tensor(X_np, dtype=torch.float32)

    adj = build_knn_graph(X, KNN_K, t=HEAT_KERNEL_T)
    edge_index = torch.tensor(np.vstack(adj.nonzero()), dtype=torch.long)

    idx = np.arange(X.shape[0])
    train_idx, temp_idx = train_test_split(idx, test_size=0.2, random_state=seed_model)
    val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=seed_model)

    def get_mask(idxs):
        mask = torch.zeros(X.shape[0], dtype=torch.bool)
        mask[idxs] = True
        return mask

    data = Data(
        x=X.to(DEVICE),
        y=Y_raw.to(DEVICE),
        edge_index=edge_index.to(DEVICE)
    )
    masks = {k: get_mask(v).to(DEVICE) for k, v in zip(['train', 'val', 'test'], [train_idx, val_idx, test_idx])}

    model = GraphSAGENet(X.size(1), SAGE_HIDDEN_DIM, Y_raw.size(1)).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=200)
    criterion = torch.nn.MSELoss()

    best_val = np.inf
    patience_counter = 0

    for epoch in range(EPOCHS):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[masks['train']], data.y[masks['train']])
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_loss = criterion(model(data.x, data.edge_index)[masks['val']], data.y[masks['val']])
        scheduler.step(val_loss)

        if val_loss < best_val:
            best_val = val_loss
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                break

    model.load_state_dict(best_state)
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        y_true = data.y

    r2 = r2_score(y_true[masks['test']].cpu().numpy(), out[masks['test']].cpu().numpy())
    mse = mean_squared_error(y_true[masks['test']].cpu().numpy(), out[masks['test']].cpu().numpy())

    return r2, mse

# ------------------------------------
# Main Execution
# ------------------------------------
if __name__ == "__main__":
    df = pd.read_csv("1000_cylindrical_anomalies.csv")
    X_raw = torch.tensor(df.iloc[:, :22].values, dtype=torch.float32)
    Y_raw = torch.tensor(df.iloc[:, 22:23].values, dtype=torch.float32)

    r2_scores = []
    mse_scores = []

    for seed in SEEDS:
        r2, mse = run_single_seed(X_raw, Y_raw, seed_model=seed)
        r2_scores.append(r2)
        mse_scores.append(mse)
        print(f"Seed {seed} -> R²: {r2:.4f}, MSE: {mse:.4f}")

    print("\nAverage Evaluation Metrics Across Seeds:")
    print(f"Mean R²: {np.mean(r2_scores):.4f}, Std R²: {np.std(r2_scores):.4f}")
    print(f"Mean MSE: {np.mean(mse_scores):.4f}, Std MSE: {np.std(mse_scores):.4f}")


Seed 8 -> R²: 0.9603, MSE: 0.1896
Seed 74 -> R²: 0.9576, MSE: 0.1791
Seed 88 -> R²: 0.9701, MSE: 0.1332
Seed 94 -> R²: 0.9537, MSE: 0.1486
Seed 111 -> R²: 0.9680, MSE: 0.1356

Average Evaluation Metrics Across Seeds:
Mean R²: 0.9619, Std R²: 0.0062
Mean MSE: 0.1572, Std MSE: 0.0230
